This script reads and saves all US POI data

In [1]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
from tqdm import tqdm
import io
import os
from global_variables import *

In [3]:
output_path_parks = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_parks.csv.gz'
parks = pd.read_csv(output_path_parks)

In [25]:
poi_csv_path = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_poi_with_store_id.csv.gz'
poi = pd.read_csv(poi_csv_path) 

In [26]:
t = poi.head()

In [13]:
# herman1 = poi[(poi['STREET_ADDRESS'] == '1500 Hermann Dr') & (poi['POSTAL_CODE'] == 77004.0)]

In [2]:
output_path_parks = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_parks.csv.gz'

output_path_hospitals = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_hospitals.csv.gz'

output_path_pharmacies = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_pharmacies.csv.gz'

output_path_zoos = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_zoos.csv.gz'

In [15]:
zoos = poi[poi['SUB_CATEGORY']== "Zoos and Botanical Gardens"]
with gzip.open(output_path_zoos, 'wt', encoding='utf-8') as gzipped_file:
    zoos.to_csv(gzipped_file, index=False)

In [16]:
len(zoos)

13971

In [4]:
parks = poi[poi['SUB_CATEGORY']== "Nature Parks and Other Similar Institutions"]

with gzip.open(output_path_parks, 'wt', encoding='utf-8') as gzipped_file:
    parks.to_csv(gzipped_file, index=False)

In [11]:
hospitals = poi[poi['SUB_CATEGORY'].isin(['Offices of Physicians (except Mental Health Specialists)','General Medical and Surgical Hospitals','Specialty (except Psychiatric and Substance Abuse) Hospitals','Outpatient Care Centers']) | poi['TOP_CATEGORY'].isin(['Outpatient Care Centers'])]

with gzip.open(output_path_hospitals, 'wt', encoding='utf-8') as gzipped_file:
    hospitals.to_csv(gzipped_file, index=False)

In [14]:
pharmacies = poi[poi['SUB_CATEGORY'].isin(['Pharmacies and Drug Stores'])]

with gzip.open(output_path_pharmacies, 'wt', encoding='utf-8') as gzipped_file:
    pharmacies.to_csv(gzipped_file, index=False)

In [18]:
parks = pd.read_csv(output_path_parks)
hospitals = pd.read_csv(output_path_hospitals)
pharmacies = pd.read_csv(output_path_pharmacies)

In [23]:
zoos = pd.read_csv(output_path_zoos)
zoos.to_parquet('C:/Users/dratnadiwakara2/Downloads/temp_dewey/zoos.parquet')

In [6]:
date_start = '2022-04-01'
date_end = '2022-08-31'

file_suffix = date_start.replace('-', '')+"_"+date_end.replace('-', '')
file_suffix

In [7]:
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/zoos_'+file_suffix+'.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
                       params={'page': page,
                               'partition_key_after': date_start,   # optionally set date value here
                               'partition_key_before': date_end}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['PLACEKEY'].isin(zoos['PLACEKEY'])  ]
    
        selected_columns = ['PLACEKEY', 'LOCATION_NAME', 'STREET_ADDRESS','POSTAL_CODE','REGION','DATE_RANGE_START','DATE_RANGE_END','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','VISITS_BY_DAY','DISTANCE_FROM_HOME','MEDIAN_DWELL']
        filtered_df = filtered_df[selected_columns]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1
        
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

1


In [16]:
df = pd.read_csv(ft_data_path)
df = df.dropna(subset=['RAW_VISIT_COUNTS'], how='any')


In [18]:
df2 = df[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']]
df2.to_parquet(ft_data_path+"_visits.parquet", index=False)

In [19]:
import json
header_option_visit_summary_file = True 
visit_summary_file = ft_data_path+"_visits_summary.csv"

with tqdm(total=len(df)) as pbar:
    
    for index in range(0, len(df)):
        try:
            temp = df.loc[index]
    
            data = json.loads(temp['VISITS_BY_DAY'])
            data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
            data['PLACEKEY'] = temp['PLACEKEY']
            data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])
            
            if os.path.isfile(visit_summary_file):
                header_option_visit_summary_file = False  
        
            data.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        except:
            pass
        
        pbar.update(1)

100%|██████████| 40168/40168 [01:54<00:00, 352.15it/s]


In [20]:
df = pd.read_csv(visit_summary_file,sep="|")  


In [21]:
df.to_parquet(visit_summary_file+".parquet", index=False)

In [11]:

with gzip.open(ft_data_path+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)

40168

In [ ]:
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_ft_data'+file_suffix+'.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
                       params={'page': page,
                               'partition_key_after': date_start,   # optionally set date value here
                               'partition_key_before': date_end}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['PLACEKEY'].isin(parks['PLACEKEY']) |
                         df['PLACEKEY'].isin(hospitals['PLACEKEY']) |
                         df['PLACEKEY'].isin(pharmacies['PLACEKEY'])  ]
    
        selected_columns = ['PLACEKEY', 'LOCATION_NAME', 'STREET_ADDRESS','POSTAL_CODE','REGION','DATE_RANGE_START','DATE_RANGE_END','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','VISITS_BY_DAY','DISTANCE_FROM_HOME','MEDIAN_DWELL']
        filtered_df = filtered_df[selected_columns]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1
        
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

In [22]:
df = pd.read_csv(ft_data_path)  

with gzip.open(ft_data_path+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)

C:\Users\dratnadiwakara2\AppData\Local\Temp\ipykernel_27448\2720968004.py:1: DtypeWarning: Columns (3,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ft_data_path)


# Clean Visits Data

In [4]:
import json
visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_month_visit_summary'+file_suffix+'.csv'
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_month_visits'+file_suffix+'.csv'
df = pd.read_csv("C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/parks_hospitals_pharmacies_ft_data20230101_20240131.csv.gz")

C:\Users\dratnadiwakara2\AppData\Local\Temp\ipykernel_26500\2865106958.py:4: DtypeWarning: Columns (3,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/parks_hospitals_pharmacies_ft_data20230101_20240131.csv.gz")


In [7]:
parks = pd.read_csv(output_path_parks)

In [11]:
herman = df[(df['STREET_ADDRESS'] == '1500 Hermann Dr') & (df['POSTAL_CODE'] == 77004.0)]


In [42]:
df2 = df[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']]
with gzip.open(visits_file+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df2.to_csv(gzipped_file, index=False)

In [ ]:
# temp = df.loc[0]
# data = json.loads(temp['VISITS_BY_DAY'])
# data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
# data['PLACEKEY'] = temp['PLACEKEY']
# data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])

In [44]:

header_option_visit_summary_file = True 

with tqdm(total=len(df)) as pbar:
    
    for index in range(0, len(df)):
        try:
            temp = df.loc[index]
    
            data = json.loads(temp['VISITS_BY_DAY'])
            data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
            data['PLACEKEY'] = temp['PLACEKEY']
            data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])
            
            if os.path.isfile(visit_summary_file):
                header_option_visit_summary_file = False  
        
            data.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        except:
            pass
        
        pbar.update(1)

100%|██████████| 13549754/13549754 [7:59:00<00:00, 471.45it/s]  


In [47]:
df = pd.read_csv(visit_summary_file,sep="|")   

In [51]:
with gzip.open(visit_summary_file+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)

In [52]:
df.head()

,day_no,no_visits,PLACEKEY,DATE_RANGE_START
0,1,0,222-223@8dj-mps-9cq,2023-01-01
1,2,5,222-223@8dj-mps-9cq,2023-01-01
2,3,3,222-223@8dj-mps-9cq,2023-01-01
3,4,5,222-223@8dj-mps-9cq,2023-01-01
4,5,5,222-223@8dj-mps-9cq,2023-01-01
